# Import Necessary Libraries

In [ ]:
import os
import pickle

import numpy as np
from scipy.sparse import lil_matrix
import cvxpy as cp

# Data Processing for Network Cascades and Adjacency Matrices

This code consists of three functions designed to process and interpret data related to network cascades and adjacency matrices. These functions are crucial for understanding network dynamics and structures in various applications such as social network analysis, information dissemination, and epidemiological studies.

## Function: `create_cascades`

### Purpose
- Generates a matrix representing cascades in a network based on input data.

## Function: `create_adj_matrix`

### Purpose
- Creates an adjacency matrix for a network based on input data.


In [ ]:
def read_variable_columns(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            columns = line.strip().split(',')
            data.append(columns)
    return data


def create_cascades(filename, num_nodes):

    # check if the file exists
    if not os.path.exists(filename):
        raise FileNotFoundError("File not found")

    # read the file
    v = read_variable_columns(filename)

    # initialize C (The cascades) with -1
    C = - np.ones((len(v), num_nodes), dtype=float)

    # loop through each row in v
    for i in range(len(v)):
        if int(v[i][0]) < num_nodes:
            C[i, int(v[i][0])] = v[i][1]

        j = 2
        while j < len(v[i]) and int(v[i][j]) > -1:
            if int(v[i][j]) < num_nodes:
                C[i, int(v[i][j])] = v[i][j+1]
            j += 2

    return C


def create_adj_matrix(filename, num_nodes):
    # initialize the adjacency matrix with zero
    A = np.zeros((num_nodes, num_nodes), dtype=float)

    # check if file exists
    if not os.path.exists(filename):
        raise FileNotFoundError("File not found")

    # read the file
    v = read_variable_columns(filename)
    v = v[:num_nodes]

    # loop through each row in v to populate the adjacency matrix
    for i in range(len(v)):
        if int(v[i][0]) < num_nodes and int(v[i][1]) < num_nodes:
            A[int(v[i][0]), int(v[i][1])] = v[i][2]

    return A

# Network Estimation Based on Diffusion Models

This code comprises three functions that collectively estimate a network's structure by considering different diffusion models. 

## Function: `add_log_survival`

### Purpose
- Updates the log-survival function based on the diffusion model.

## Function: `add_constraints`

### Purpose
- Adds constraints to a convex optimization problem based on the type of diffusion.

## Function: `estimate_network`

### Purpose
- Estimates the network structure using a convex optimization approach.

### Process
1. **Initialization:**
   - Sets up various matrices (`A_potential`, `A_bad`, `A_hat`) and initializes a counter for cascades per node.
2. **Cascade Processing:**
   - Iterates through each cascade, updating `A_potential` and `A_bad` using `add_log_survival`.
3. **Convex Optimization:**
   - For each node, sets up and solves a convex optimization problem to estimate network connections.
   - Uses `cp.Variable` to create optimization variables.
   - Adds constraints and objectives specific to the diffusion model and solves the problem.
4. **Result Compilation:**
   - Aggregates results into `A_hat`, representing the estimated network structure.
   - Returns the estimated network `A_hat` and the total objective function value `total_obj`.


In [ ]:
def add_log_survival(A, i, j, diff, type_diffusion):
    if type_diffusion == 'exp':
        A[i, j] += diff
    elif type_diffusion == 'pl':
        A[i, j] += np.log(diff)
    elif type_diffusion == 'rayleigh':
        A[i, j] += 0.5 * (diff) ** 2


def add_constraints(constraints, a_hat, t_hat, c_act, val, idx_ord, cidx, type_diffusion):
    if type_diffusion == 'exp':
        constraints.append(t_hat[c_act] == cp.sum(a_hat[idx_ord[:cidx[0]]]))
    elif type_diffusion == 'pl':
        tdifs = 1.0 / (val[cidx[0]] - val[:cidx[0]])
        indv = np.where(tdifs < 1)[0]
        tdifs = tdifs[indv]
        constraints.append(t_hat[c_act] <= cp.sum(
            tdifs * a_hat[idx_ord[indv]]))
    elif type_diffusion == 'rayleigh':
        tdifs = (val[cidx[0]] - val[:cidx[0]])
        constraints.append(t_hat[c_act] <= cp.sum(
            tdifs * a_hat[idx_ord[:cidx[0]]]))


def estimate_network(A, C, num_nodes, horizon, type_diffusion):
    num_cascades = np.zeros(num_nodes)
    A_potential = lil_matrix(np.zeros(A.shape))
    A_bad = lil_matrix(np.zeros(A.shape))
    A_hat = lil_matrix(np.zeros(A.shape))
    total_obj = 0

    for c in range(C.shape[0]):
        idx = np.where(C[c, :] != -1)[0]  # used nodes
        val = np.sort(C[c, idx])
        order = np.argsort(val)

        for i in range(1, len(val)):
            num_cascades[idx[order[i]]] += 1
            for j in range(i):
                diff = val[i] - val[j]
                add_log_survival(
                    A_potential, idx[order[j]], idx[order[i]], diff, type_diffusion)

        for j in range(num_nodes):
            if j not in idx:
                for i in range(len(val)):
                    diff = horizon - val[i]
                    add_log_survival(
                        A_bad, idx[order[i]], j, diff, type_diffusion)

    # convex program per column
    for i in range(num_nodes):
        if num_cascades[i] == 0:
            continue
        print(f'Processing node {i}...')
        a_hat = cp.Variable(num_nodes)
        t_hat = cp.Variable(int(num_cascades[i]))
        obj = 0

        potential_indices = A_potential[:, i].nonzero()[0]
        bad_indices = A_bad[:, i].nonzero()[0]

        constraints = [a_hat[potential_indices] >= 0]

        for j in potential_indices:
            obj += -a_hat[j] * (A_potential[j, i] + A_bad[j, i])

        c_act = 0
        for c in range(C.shape[0]):
            idx = np.where(C[c, :] != -1)[0]  # used nodes
            val = np.sort(C[c, idx])
            order = np.argsort(val)
            idx_ord = idx[order]
            cidx = np.where(idx_ord == i)[0]

            if cidx.size > 0 and cidx[0] > 0:
                add_constraints(constraints, a_hat, t_hat, c_act,
                                val, idx_ord, cidx, type_diffusion)
                obj += cp.log(t_hat[c_act])
                c_act += 1

        constraints += [a_hat >= 0]

        problem = cp.Problem(cp.Maximize(obj), constraints)
        problem.solve(solver=cp.CLARABEL, verbose=False)

        total_obj += problem.value
        A_hat[:, i] = a_hat.value

    return A_hat, total_obj



# Function: `netrate`

## Overview
The `netrate` function is the core of the network estimation process, integrating various components to estimate a network's structure based on diffusion models. It involves reading and processing network and cascades data, estimating the network structure, and evaluating its accuracy.

## Parameters
- `network`: Path to the network file.
- `cascades`: Path to the cascades data file.
- `horizon`: Time horizon for the cascades.
- `type_diffusion`: Type of diffusion model used (e.g., 'exp', 'pl', 'rayleigh').
- `num_nodes`: Number of nodes in the network.

## Process

### 1. Initialization
- Sets a minimum tolerance level `min_tol` for the network estimation.
- Initializes an array `pr` for precision and recall values.

### 2. Reading Ground-Truth Network
- Reads the ground-truth network from `network` using the `create_adj_matrix` function.

### 3. Reading Cascades Data
- Reads and processes the cascades data from `cascades` using the `create_cascades` function.

### 4. Estimating the Network
- Calls `estimate_network` with the ground-truth network `A`, cascades `C`, and other parameters to estimate the network structure.
- Receives the estimated network `A_hat` and the total objective function value `total_obj`.

### 5. Accuracy Evaluation
- If a ground-truth network is available, calculates Mean Absolute Error (MAE), precision, and recall to evaluate the accuracy of `A_hat`.
- Compares `A_hat` with the ground-truth `A` to compute these metrics.
- Constructs binary versions of `A` and `A_hat` to calculate precision and recall.

### 6. Result Saving
- Extracts the network name from the `network` path.
- Saves the results (estimated network, MAE, precision, recall, total objective) to a file in the `Result` directory.

## Returns
- `A_hat`: The estimated network matrix.
- `total_obj`: The total objective function value from the network estimation.
- `pr`: Precision and recall values, if applicable.
- `mae`: Mean Absolute Error, if applicable.


In [ ]:
def netrate(network, cascades, horizon, type_diffusion, num_nodes):
    min_tol = 1e-4

    pr = np.zeros(2)

    print('Reading ground-truth...')
    A = create_adj_matrix(network, num_nodes)

    print('Reading cascades...')
    C = create_cascades(cascades, num_nodes)

    print('Building data structures...')
    A_hat, total_obj = estimate_network(
        A, C, num_nodes, horizon, type_diffusion)

    if os.path.exists(network):
        non_zero_mask = (A != 0)
        A_non_zero = A[non_zero_mask]
        A_hat_non_zero = A_hat[non_zero_mask]
        mae = np.mean(np.abs(A_hat_non_zero - A_non_zero) / A_non_zero)

        # Compute precision and recall
        A_hat_binary = A_hat > min_tol
        A_binary = A > min_tol

        # Element-wise logical AND using the multiply method
        intersection = A_hat_binary.multiply(A_binary)
        recall = intersection.sum() / A_binary.sum()
        precision = intersection.sum() / A_hat_binary.sum()
        pr = np.array([precision, recall])
    else:
        mae = None
        pr = None

    network = network.split('/')[-1].split('.')[0]

    # Saving the results
    with open(f'Result/solution-{network}.pkl', 'wb') as f:
        pickle.dump({'A_hat': A_hat, 'mae': mae,
                    'pr': pr, 'total_obj': total_obj}, f)

    return A_hat, total_obj, pr, mae



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

network_file = '../Data/kronecker-core-periphery-n1024-h10-r0_01-0_25-network.txt'
cascades_file = '../Data/kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt'

# network_file = '../Data/netinf-512-network.txt'
# cascades_file = '../Data/netinf-512-cascades.txt'


horizon = 20
type_diffusion = 'exp'
num_nodes = 200  # number of nodes in Kronecker graph
# num_nodes = 512 # number of nodes in NetInf graph


A_hat, total_obj, pr, mae = netrate(
    network_file, cascades_file, horizon, type_diffusion, num_nodes)

# convert A_hat to matrix
A_hat = A_hat.toarray()

A = create_adj_matrix(network_file, num_nodes)

# print the results
print(f'Precision: {pr[0]}')
print(f'Recall: {pr[1]}')
print(f'Mean Absolute Error: {mae}')
print(f'Total Objective: {total_obj}')

# Save the results
network = network_file.split('/')[-1].split('.')[0]

# make the Result/{network} directory if it does not exist
if not os.path.exists(f'Result/{network}'):
    os.mkdir(f'Result/{network}')


with open(f'solution.pkl', 'wb') as f:
    pickle.dump({'A_hat': A_hat, 'mae': mae,
                'pr': pr, 'total_obj': total_obj}, f)


# save the plots, beautify the plots with seaborn for the graphs. note that the graphs are relatively large
G = nx.from_numpy_array(A_hat)
nx.draw(G, with_labels=True, font_weight='bold', node_size=100, node_color='skyblue', font_size=8, edge_color='gray',
        linewidths=0.5, width=0.5, alpha=0.7, pos=nx.spring_layout(G), arrowsize=10, arrowstyle='->', connectionstyle='arc3, rad = 0.1', edge_cmap=plt.cm.Blues)
plt.savefig(f'Result/{network}/A_hat_{num_nodes}.png', dpi=300)

G = nx.from_numpy_array(A)
nx.draw(G, with_labels=True, font_weight='bold', node_size=100, node_color='skyblue', font_size=8, edge_color='gray',
        linewidths=0.5, width=0.5, alpha=0.7, pos=nx.spring_layout(G), arrowsize=10, arrowstyle='->', connectionstyle='arc3, rad = 0.1', edge_cmap=plt.cm.Blues)
plt.savefig(f'Result/{network}/A_{num_nodes}.png', dpi=300)

